In [0]:
from pyspark.sql.functions import *

In [0]:
from pyspark.sql import SparkSession

# Initialize Spark Session (if not already created)
spark = SparkSession.builder.appName("StocksDF").getOrCreate()

# Data from the image
data = [
    ("Apple", "Buy", 1, 1500),
    ("Tesla", "Buy", 2, 1200),
    ("Apple", "Sell", 5, 5000),
    ("Samsung", "Buy", 17, 20000),
    ("Tesla", "Sell", 3, 1300),
    ("Tesla", "Buy", 4, 1500),
    ("Tesla", "Sell", 5, 1100),
    ("Tesla", "Buy", 6, 1400),
    ("Samsung", "Sell", 29, 15000),
    ("Tesla", "Sell", 10, 1200)
]

# Column names
columns = ["stock_name", "operation", "operation_day", "price"]

# Create DataFrame
df = spark.createDataFrame(data, columns)

# Show DataFrame
df.createOrReplaceTempView("stocks")



In [0]:
%sql
with cte as (
  select stock_name,
  sum(case when operation = 'Buy' then price end) as Buy_price,
  sum(case when operation = 'Sell' then price end) as Sell_price from stocks
  group by stock_name
)
select stock_name, (Sell_price - Buy_price) as capital_gain_loss from cte

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DateType
from pyspark.sql.functions import *

# Create Spark session (if not already created)
spark = SparkSession.builder.appName("EventDF").getOrCreate()

# Data from image
data = [
    ("fail", "2020-01-04"),
    ("success", "2020-01-01"),
    ("success", "2020-01-03"),
    ("success", "2020-01-06"),
    ("fail", "2020-01-05"),
    ("success", "2020-01-02")
]

# Define schema
schema = StructType([
    StructField("event", StringType(), True),
    StructField("event_dt", StringType(), True)  # Initially string
])

# Create DataFrame
df = spark.createDataFrame(data, schema)

# Convert to Date type (recommended in real projects)
df = df.withColumn("event_dt", to_date("event_dt", "yyyy-MM-dd"))

# Show result
df.display()
df.createOrReplaceTempView('events')

In [0]:
%sql
with cte as (
select *,
date_diff(event_dt, min(event_dt)over(order by event_dt)) - row_number() over (partition by event order by event_dt)as n,
date_add(event_dt, -(row_number() over (partition by event order by event_dt))) as grp
 from events 
)
select * from cte
-- select event, min(event_dt) as start_date, max(event_dt) as end_date from cte
-- group by event, grp
-- order by start_date

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

# Create Spark session (if not already created)
spark = SparkSession.builder.appName("TeamsDF").getOrCreate()

# Data from image
data = [
    ("India", "Australia", "India"),
    ("India", "England", "England"),
    ("SouthAfrica", "India", "India"),
    ("Australia", "England", None),   # NULL value
    ("England", "SouthAfrica", "SouthAfrica"),
    ("Australia", "India", "Australia")
]

# Define schema
schema = StructType([
    StructField("Team_1", StringType(), True),
    StructField("Team_2", StringType(), True),
    StructField("Result", StringType(), True)
])

# Create DataFrame
df = spark.createDataFrame(data, schema)

# Show DataFrame
df.display()
df.createOrReplaceTempView('teams')


In [0]:
%sql
select Matches, Matches_won, Matches_lost, (Matches - (Matches_won + Matches_lost)) as Matches_tied from (
select team, sum(Matches) as Matches, sum(Matches_won) as Matches_won, sum(Matches_lost) as Matches_lost from (
select Team_1 as team, count(*) as Matches,
sum(case when Result = Team_1 then 1 else 0 end) as Matches_won,
sum(case when Result != Team_1 then 1 else 0 end) as Matches_lost
from teams
 group by 1
union all
select Team_2 as team, count(*) as Matches,
sum(case when Result = Team_2 then 1 else 0 end) as Matches_won ,
sum(case when Result != Team_2 then 1 else 0 end) as Matches_lost 
from teams
group by 1
)a
group by 1
)b



In [0]:
students = {"Ram": "+91 9988776655", "Sita": "+92 7766554422", "John": "+88 654567898", "Anjali": "+1 9598762345"}

data = [{"name": name, "mobile_no": mobile_no} for name, mobile_no in students.items()]
df = spark.createDataFrame(data)
display(df)

In [0]:
df.createOrReplaceTempView('students')

In [0]:
%sql
select *, concat(split(mobile_no, ' ')[0], repeat('*', length(split(mobile_no, ' ')[1])-4), right(split(mobile_no, ' ')[1],4) ) as new_phn from students

In [0]:
df

In [0]:
final_df = df.withColumn('new_phn', concat(split(col('mobile_no'), ' ')[0], repeat('*', length(split(col('mobile_no'), ' ')[1])-4), right(split(col('mobile_no'), ' ')[1],4) ))
final_df.display()

In [0]:
from pyspark.sql import functions as F

# 1. Split the mobile number into an array [country_code, number]
# 2. Calculate the length of the number part
# 3. Mask the middle of the number part
# 4. Concatenate it back together

df_new = df.withColumn("split_phn", F.split(F.col("mobile_no"), " ")) \
           .withColumn("country_code", F.col("split_phn").getItem(0)) \
           .withColumn("phone_part", F.col("split_phn").getItem(1)) \
           .withColumn("mask_length", F.length(F.col("phone_part")) - 4) \
           .withColumn("new_phn", F.concat(
               F.col("country_code"),
               F.repeat(F.lit("*"), F.col("mask_length")),
               F.substring(F.col("phone_part"), -4, 4)
           )) \
           .drop("split_phn", "country_code", "phone_part", "mask_length")

df_new.display()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, DateType
from pyspark.sql.functions import to_date

spark = SparkSession.builder.getOrCreate()

data = [
    # Employee 1 → Has 3 consecutive days
    (1, "2025-01-20"),
    (1, "2025-01-21"),
    (1, "2025-01-22"),
    (1, "2025-01-24"),

    # Employee 2 → No 3 consecutive days
    (2, "2025-01-15"),
    (2, "2025-01-16"),
    (2, "2025-01-18"),

    # Employee 3 → Has 3 consecutive days
    (3, "2025-01-10"),
    (3, "2025-01-11"),
    (3, "2025-01-12"),

    # Employee 4 → Has 4 consecutive days
    (4, "2025-01-05"),
    (4, "2025-01-06"),
    (4, "2025-01-07"),
    (4, "2025-01-08"),

    # Employee 5 → Only 2 consecutive days
    (5, "2025-01-01"),
    (5, "2025-01-02"),

    # Employee 6 → Multiple streak attempts
    (6, "2025-01-01"),
    (6, "2025-01-03"),
    (6, "2025-01-04"),
    (6, "2025-01-05")
]

schema = StructType([
    StructField("emp_id", IntegerType(), True),
    StructField("login_date", DateType(), True)
])

login_df = spark.createDataFrame(data, ["emp_id", "login_date"]) \
          .withColumn("login_date", to_date("login_date"))

login_df.display()
login_df.createOrReplaceTempView("employee_logs")

In [0]:

from pyspark.sql.window import Window

wn = Window.partitionBy(col('emp_id')).orderBy(col('login_date')) 

final_df = login_df.withColumn('rn', date_sub(col('login_date'), row_number().over(wn))).groupBy('emp_id','rn').agg(count('*').alias('cnt')).filter("cnt >= 3").select('emp_id', 'rn')

final_df.display()

In [0]:
%sql
with cte as (
select emp_id, login_date,
date_add(DAY, -row_number()over(partition by emp_id order by login_date), login_date) as rn
from employee_logs
)
select * from cte
-- select emp_id, rn from cte
-- group by emp_id, rn
-- having count(*) >=3

In [0]:
from pyspark.sql.functions import col, split, concat, repeat, length, right, lit

# split_col = split(col("mobile_no"), " ")

# masked_length = length(split(col("mobile_no"), " ")[1]) - 4

final_df = df.withColumn(
    "new_phn",
    concat(
        split(col("mobile_no"), " ")[0],
        repeat(lit("*"), length(split(col("mobile_no"), " ")[1]) - 4),
        substring(split(col("mobile_no"), " ")[1], -4, 4)
    )
)

display(final_df)

In [0]:
from pyspark.sql.functions import col, split, concat, repeat, length, substring, when, greatest

# Split only once
# split_col = split(col("mobile_no"), " ")

masked_length = length(split(col("mobile_no"), " ")[1]) - 4

final_df = df.withColumn(
    "new_phn",
    concat(
        split(col("mobile_no"), " ")[0],
        repeat(col("*"), masked_length),  # <-- FIX HERE
        substring(split(col("mobile_no"), " ")[1], -4, 4)
    )
)


display(final_df)


In [0]:
final_df = df.withColumn('new_phn', concat(split(col('mobile_no'), ' ')[0], repeat('*', length(split(col('mobile_no'), ' ')[1])-4), right(split(col('mobile_no'), ' ')[1],4) ))
final_df.display()

In [0]:
data = [
    {"id": 1},
    {"id": 2},
    {"id": 3},
    {"id": 4}
]
df = spark.createDataFrame(data)
display(df)

In [0]:
from pyspark.sql.functions import *
result_df = df.withColumn('dummy', sequence(lit(1), col('id')))
result_df.show()

In [0]:
data = [
    {"id": 1, "emp_name": "CEO", "mgr_id": None},
    {"id": 2, "emp_name": "CFO", "mgr_id": 1},
    {"id": 3, "emp_name": "CTO", "mgr_id": 1},
    {"id": 4, "emp_name": "Manager1", "mgr_id": 2},
    {"id": 5, "emp_name": "Manager2", "mgr_id": 2},
    {"id": 6, "emp_name": "Lead", "mgr_id": 4},
    {"id": 7, "emp_name": "Engineer", "mgr_id": 6}
]
df = spark.createDataFrame(data)
display(df)

In [0]:
df.schema['emp_name']

In [0]:
df.createOrReplaceTempView('emp')

In [0]:
%sql

with recursive cte as (
  select emp_name, id, mgr_id, 1 as emp_level from emp where mgr_id is null
  union all
  select e.emp_name, e.id, e.mgr_id, m.emp_level+1 as emp_level from emp e join cte m on e.mgr_id = m.id
)
select emp_name, id, emp_level from cte
order by emp_level

In [0]:
data = [
    {"col1": "A", "col2":"B"},
    {"col1": "C", "col2":"D"},
    {"col1": "E", "col2":"F"},
    {"col1": "G", "col2":"H"},
    {"col1": "I", "col2":"J"}
]
df = spark.createDataFrame(data)
display(df)

In [0]:
df.createOrReplaceTempView('seating')

In [0]:
%sql

with cte as (
  select col2 as col1, col1 as col2,
  row_number() over (order by col1, col2) as rn
  from seating
)
select col1, col2,
case 
when rn%2 = 1 and rn = (select max(rn) from cte) then rn
when rn%2 = 1 then rn+1
else rn-1 end as new_rn
 from cte
 order by new_rn

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Create Spark session
spark = SparkSession.builder.appName("CreateDataFrameFromImage").getOrCreate()

# Define schema
schema = StructType([
    StructField("team", StringType(), True)
])

# Data extracted from image
data = [
'CSK',
'RCB',
'MI',
'KKR'
]

# Create DataFrame
df = spark.createDataFrame(data, schema)

# Show result
df.display()


In [0]:
%sql
with cte as (
select NUMB,
lead(NUMB,1) OVER(order by SN) as next_id,
lead(NUMB,2) OVER(order by SN) as next_next_id from serial
)
select NUMB from cte 
where NUMB != next_id and NUMB = next_next_id

In [0]:
df.createOrReplaceTempView('ipl')

In [0]:
%sql
select * from ipl

In [0]:
%sql
select concat(team1,'vs',team2) from (
select a.team as team1, b.team as team2 from
ipl a join ipl b on a.team<>b.team
)a
where team1<team2

In [0]:

original_dict = original_dict = {
    'a': 1,
    'b': 2,
    'c': 1,
    'd': 3
}

def reverse_dict(d):
    result = {}
    for k,v in d.items():
        if v in result:
            result[v].append(k)
        else:
            result[v] = [k]
    return result



print(reverse_dict(original_dict)) 

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, row_number, datediff, min
from pyspark.sql.window import Window

spark = SparkSession.builder.getOrCreate()

# Sample data
data = [
    ("u1", "2023-01-01"),
    ("u1", "2023-01-02"),
    ("u1", "2023-01-04"),
    ("u1", "2023-01-05"),
    ("u1", "2023-01-06"),
    ("u2", "2023-01-10")
]

df = spark.createDataFrame(data, ["user_id", "login_date"]) \
          .withColumn("login_date", col("login_date").cast("date"))

# Step 1: Add row_number per user ordered by date
window_spec = Window.partitionBy("user_id").orderBy("login_date")

df = df.withColumn("rn", row_number().over(window_spec))

# Step 2: Create grouping key using date - row_number
df = df.withColumn(
    "grp",
    datediff(col("login_date"), min("login_date").over(window_spec)) - col("rn"),
).withColumn('new', datediff(col("login_date"), min("login_date").over(window_spec))).withColumn('n_date', min("login_date").over(window_spec))
display(df)




In [0]:
# Step 3: Count streak length per group
streak_window = Window.partitionBy("user_id", "grp") \
                       .orderBy("login_date")

df = df.withColumn("streak", row_number().over(streak_window))

df.display()
# # Final result
# result = df.select("user_id", "login_date", "streak") \
#            .orderBy("user_id", "login_date")

result.show()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import *
# Create Spark Session (skip if already created)
spark = SparkSession.builder.appName("CreateProductDF").getOrCreate()

# Define Schema
schema = StructType([
    StructField("product_id", IntegerType(), False),
    StructField("product_name", StringType(), False),
    StructField("category", StringType(), False)
])

# Data from image
data = [
    (4, "Headphones", "Accessories"),
    (5, "Smartwatch", "Accessories"),
    (6, "Keyboard", "Accessories"),
    (7, "Mouse", "Accessories"),
    (8, "Monitor", "Accessories"),
    (1, "Laptop", "Electronics"),
    (2, "Smartphone", "Electronics"),
    (3, "Tablet", "Electronics"),
    (9, "Printer", "Electronics")
]

# Create DataFrame
df = spark.createDataFrame(data, schema)

# Show DataFrame
df.display()
df.createOrReplaceTempView('items')


In [0]:
%sql
select *, 
row_number() over(partition by category order by product_id) as rn from items

In [0]:
%sql
with cte1 as (
select *, 
row_number() over(partition by category order by product_id) as rn from items
),
cte2 as (
select *, 
row_number() over(partition by category order by product_id desc) as rn from items
)
select c2.product_id as product_id, c1.product_name as product_name, c2.category as category from cte1 c1 join cte2 c2 on c1.category = c2.category and c1.rn = c2.rn

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

# Create Spark Session (skip if already created)
spark = SparkSession.builder.appName("CityPincodeDF").getOrCreate()

# Define schema
schema = StructType([
    StructField("city_pincode", StringType(), False)
])

# Data from image
data = [
    ("Mumbai400001",),
    ("Delhi110001",),
    ("Bengaluru560001",),
    ("Kolkata700001",),
    ("Chennai600001",),
    ("Hyderabad500001",),
    ("Ahmedabad380001",),
    ("Pune411001",),
    ("Jaipur302001",),
    ("Lucknow226001",)
]

# Create DataFrame
df = spark.createDataFrame(data, schema)

# Show DataFrame
df.display()

df.createOrReplaceTempView('pincode')


In [0]:
%sql
select *,
regexp_replace(city_pincode, '[^A-za-z]', '') as City,
regexp_replace(city_pincode, '[^0-9]', '') as Pincode,
regexp_substr(city_pincode, '^[A-Za-z]+') as City1,
regexp_substr(city_pincode, '[0-9]+') as pin1,
regexp_extract(city_pincode, '^[A-Za-z]+', 0) as City2,
regexp_extract(city_pincode, '[0-9]+', 0) as pin2
from pincode

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW transactions AS
SELECT * FROM (
  VALUES
    ('2025-06-01',602),('2025-06-02',935),('2025-06-02',205),('2025-06-03',1360),('2025-06-04',770),
    ('2025-06-05',606),('2025-06-06',571),('2025-06-07',1200),('2025-06-08',520),
    ('2025-06-09',1114),('2025-06-10',621),('2025-06-11',1373),('2025-06-12',1493),
    ('2025-06-13',1380),('2025-06-14',1449),('2025-06-15',962),('2025-06-16',793),
    ('2025-06-17',1410),('2025-06-18',704),('2025-06-19',1470),('2025-06-20',1031),
    ('2025-06-21',1186),('2025-06-22',671),('2025-06-23',1307),('2025-06-24',1198),
    ('2025-06-25',615),('2025-06-26',1212),('2025-06-27',1122),('2025-06-28',920),
    ('2025-06-29',919),('2025-06-30',1394), ('2025-06-30',1302),('2025-07-01',1123),('2025-07-02',1073),
    ('2025-07-03',1021),('2025-07-04',1438),('2025-07-05',1324),('2025-07-06',764),
    ('2025-07-07',1431),('2025-07-08',1280),('2025-07-09',1444),('2025-07-10',1130),
    ('2025-07-11',1409),('2025-07-12',656),('2025-07-13',643),('2025-07-14',905),
    ('2025-07-15',1332),('2025-07-16',894),('2025-07-17',1434),('2025-07-18',719),
    ('2025-07-19',787),('2025-07-20',824),('2025-07-21',1467),('2025-07-22',1111),
    ('2025-07-23',1060),('2025-07-24',720),('2025-07-25',1296),('2025-07-26',823),
    ('2025-07-27',1480),('2025-07-28',879),('2025-07-29',1341),('2025-07-30',1037),
    ('2025-07-31',1221),('2025-08-01',916),('2025-08-02',606),('2025-08-03',1094),
    ('2025-08-04',1045),('2025-08-05',1192), ('2025-08-05',1000),('2025-08-06',1214),('2025-08-07',1343),
    ('2025-08-08',792),('2025-08-09',859),('2025-08-10',1141),('2025-08-11',673),
    ('2025-08-12',1042),('2025-08-13',1480),('2025-08-14',1368),('2025-08-15',1195),
    ('2025-08-16',1493),('2025-08-17',1434),('2025-08-18',1248),('2025-08-19',1154),
    ('2025-08-20',1322),('2025-08-21',680),('2025-08-22',1163),('2025-08-23',901),
    ('2025-08-24',923),('2025-08-25',1410),('2025-08-26',608),('2025-08-27',502),
    ('2025-08-28',642),('2025-08-29',1188)
) AS t(txn_date, amount);

In [0]:
%sql
with cte as (
select txn_date, sum(amount) as total from transactions group by 1),
cte2 as (
select *,
sum(total)over(order by txn_date ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS 30_day_rolling_sum
from cte
)
select *,
coalesce(round(100 * (30_day_rolling_sum - lag(30_day_rolling_sum)over(order by txn_date))/ (lag(30_day_rolling_sum)over(order by txn_date)),2), 0) as percent_change 
from cte2

In [0]:
def even_odd(lst):
    even_count = sum(1 for x in lst if x%2 == 0)
    odd_count = len(lst) - even_count

    return even_count == odd_count


print(even_odd([1,2,3,4,5,6,7,8]))